### Development Notebook for  `build_groundtruth_tf` function 
- Setup model, load weights and data generators 
- get items from data generator
- pass through network

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.utils       import mask_string, parse_image_meta, apply_box_deltas_tf
from mrcnn.visualize   import display_gt_bboxes, display_roi_proposals, plot_gaussian, plot_gaussian2
from mrcnn.visualize   import display_gt_bboxes, display_roi_proposals
import mrcnn.visualize as visualize
from mrcnn.prep_notebook import prep_oldshapes_dev

model, dataset_train, train_generator, config = prep_oldshapes_dev(init_with = 'last', FCN_layers = True)
model_info = [model, config, dataset_train, train_generator]

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
model.layer_info()
# model.keras_model.outputs[0].name

In [ ]:
# model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
model_output = get_layer_output_1(model.keras_model, train_batch_x, [4,5,6,7,15,16,17,18,19,20,21,22,23], 1)


In [ ]:
print(len(model_output))

output_rois               = model_output[0]          # layer:  4   shape: (5, 32, 4)
target_class_ids          = model_output[1]          # layer:  5   shape: (5, 32)
target_bbox_deltas        = model_output[2]          # layer:  6   shape: (5, 32, 4)
roi_gt_boxes              = model_output[3]          # layer:  7   shape: (5, 32, 4)

pred_heatmap_norm         = model_output[4]          # layer: 15   shape: (5, 128, 128, 4)
gt_heatmap_norm           = model_output[5]          # layer: 16   shape: (5, 128, 128, 4)

pred_heatmap_scores       = model_output[6]          # layer: 17   shape: (5, 4, 32, 10)
gt_heatmap_scores         = model_output[7]          # layer: 18   shape: (5, 4, 100, 10)

pred_tensor               = model_output[8]          # layer: 19   shape: (5, 4, 32, 6)
gt_tensor                 = model_output[9]          # layer: 20   shape: (5, 4, 100, 6)

fcn_heatmap               = model_output[10]          # layer: 21   shape: (5, 128, 128, 4)
fcn_heatmap_norm          = model_output[11]          # layer: 22   shape: (5, 128, 128, 4)
fcn_scores                = model_output[12]          # layer: 23   shape: (5, 4, 32, 14)

for i in model_output:
    print( i.shape)

In [ ]:
input_image      =  train_batch_x[0]
input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes    = train_batch_x[5]
input_gt_masks     = train_batch_x[6]
print(' Input image shape is :', input_image.shape)
h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
input_normlzd_gt_bboxes = tf.identity(input_gt_bboxes / [h,w,h,w])

# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)
print(' input_normlzd_gt_bboxes    ', input_normlzd_gt_bboxes.shape)


####  display Pred_Tensor, Pred_heatmap, mrcnn_class

In [ ]:
np.set_printoptions(precision=4, threshold=None, linewidth=120, suppress=True)
img = 0

# print(KB.int_shape(output_rois))
# print(output_rois[img,:15]*[128, 128,128,128])
# print(input_gt_class_ids[0])

# print(' Pred_tensor')
# print(pred_tensor.shape)
# print(pred_tensor[img,:,:10])

# print(' gt_tensor')
# print(gt_tensor.shape)
# print(gt_tensor[img,:,:10])

# print(' output_rois')
# print(output_rois.shape)
# print(output_rois[img,:15] * [128, 128,128,128])

print(' target_class_ids')
print(target_class_ids.shape)
print(target_class_ids[img,:15])


print(' roi_gt_boxes')
print(roi_gt_boxes.shape)
print(roi_gt_boxes[img,:15] * [128, 128,128,128])

print(' Pred Heatmap Scores')
print(pred_heatmap_scores.dtype)
print(pred_heatmap_scores[img,:,:10])

# print(' FCN Scores')
# print(fcn_scores.dtype)
# print(fcn_scores[img,:,:10, 4:])

# img = 2
# max_score = np.max(mrcnn_class, axis = -1)
# max_class = np.argmax(mrcnn_class, axis = -1)
# # print(' output_rois[',img,'] \n', output_rois[1]*[128,128,128,128])
# print('max class shape:',max_class.shape, 'max score shape: ',max_score.shape)
# print('max class[',img,']\n',max_class[img])
# print('max score[',img,']\n',max_score[img])
# print('mrcnn class.shape ',mrcnn_class.shape)
# print('mrcnn_class[',img,',:]\n',mrcnn_class[img,:])
# print(output_rois[1])
 

##  `current build_ground_truth_tf ()` 

In [ ]:
# sess = KB.get_session()
##----------------------------------------------------------------------------------------------------------------------          
## 
##----------------------------------------------------------------------------------------------------------------------          
def build_ground_truth_tf(gt_class_ids, norm_gt_bboxes, config):
    # // pass model to TensorBuilder
    batch_size      = config.BATCH_SIZE
    num_classes     = config.NUM_CLASSES
    h, w            = config.IMAGE_SHAPE[:2]
    num_bboxes      = KB.int_shape(gt_bboxes)[1]
    gt_bboxes       = norm_gt_bboxes * tf.constant([h,w,h,w], dtype = tf.float32)   
    print()
    # num of bounding boxes is determined by bbox_list.shape[1] instead of config.DETECTION_MAX_INSTANCES
    # use of this routine for both input_gt_boxes, and target_gt_deltas
    if tf.equal(num_bboxes == tf.constant(config.DETECTION_MAX_INSTANCES)):
        tensor_name = "gt_tensor_max"
    else:
        tensor_name = "gt_tensor"
        
    print('\n')
    print('  > BUILD_GROUND TRUTH_TF()' )
    print('    gt_class_ids shape : ', gt_class_ids.get_shape())
    print('    gt_bboxes.shape    : ', gt_bboxes.get_shape() )

    #---------------------------------------------------------------------------
    # use the argmaxof each row to determine the dominating (predicted) class
    # mask identifies class_ids > 0 
    #---------------------------------------------------------------------------
    gt_classes_exp = tf.to_float(tf.expand_dims(gt_class_ids ,axis=-1))
    print('    gt_classes_exp shape ', gt_classes_exp.get_shape() )

    ones = tf.ones_like(gt_class_ids)
    zeros= tf.zeros_like(gt_class_ids)
    mask = tf.greater(gt_class_ids , 0)

    gt_scores     = tf.where(mask, ones, zeros)
    # pred_scores      = tf.reduce_max(mrcnn_class ,axis=-1, keep_dims=True)   # (32,)
    gt_scores_exp = tf.to_float(KB.expand_dims(gt_scores, axis=-1))
    print('    gt_scores_exp shape ', gt_scores_exp.get_shape())

    #---------------------------------------------------------------------------
    # create meshgrid to do something 
    #---------------------------------------------------------------------------
    batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32), 
                                         tf.range(num_bboxes, dtype=tf.int32), indexing = 'ij' )
    
    # sequence id is used to preserve the order of rois as passed to this routine
    sequence = gt_scores * (bbox_grid[...,::-1] + 1) 
    sequence = tf.to_float(tf.expand_dims(sequence, axis = -1))   

    # print('    batch_grid shape  ', batch_grid.get_shape())
    # print('    bbox_grid  shape  ', bbox_grid.get_shape())
    # print('    sequence shape    ', sequence.get_shape())

    # obsolete replaced with sequence 12-05-2018    
    # bbox_idx_zeros  = tf.zeros_like(bbox_grid)
    # bbox_idx        = tf.where(mask, bbox_grid , bbox_idx_zeros)
    # bbox_idx        = tf.to_float(tf.expand_dims(bbox_idx, axis = -1))    
    
    # 12-05-2018 (added sequecne)    gt_array  = tf.concat([gt_bboxes, gt_classes_exp, gt_scores_exp], axis=2)
    gt_array        = tf.concat([gt_bboxes, gt_classes_exp, gt_scores_exp, sequence ], axis=2)
    print('    gt_array shape :', gt_array.shape, gt_array.get_shape())

    #------------------------------------------------------------------------------
    # Create indicies to scatter rois out to multi-dim tensor by image id and class
    # resulting tensor is batch size x num_classes x num_bboxes x 7 (num columns)
    #------------------------------------------------------------------------------
    scatter_ind = tf.stack([batch_grid , gt_class_ids, bbox_grid],axis = -1)

    # obsolete 12-05-2018 - replaced num_cols with gt_array.shape[-1]
    # gt_scatter = tf.scatter_nd(scatter_ind, gt_array, [batch_size, num_classes, num_bboxes, num_cols ])
    gt_scatter = tf.scatter_nd(scatter_ind, gt_array, [batch_size, num_classes, num_bboxes, gt_array.shape[-1] ])

    # print('    scatter_ind shape ', scatter_ind.get_shape())
    # print('    gt_scatter shape ', gt_scatter.get_shape())
    
    #-------------------------------------------------------------------------------
    ## sort in each class dimension based on on sequence number (last column)
    # scatter_nd places bboxs in a sparse fashion --- this sort is to place all bboxes
    # at the top of the class bbox array
    #-------------------------------------------------------------------------------

    _ , sort_inds = tf.nn.top_k(tf.abs(gt_scatter[:,:,:,-1]), k=gt_scatter.shape[2])

    # build indexes to gather rows from pred_scatter based on sort order 
    class_grid, batch_grid, bbox_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_bboxes))
    bbox_grid_exp = tf.to_float(tf.expand_dims(bbox_grid, axis = -1))
 
    gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
    gt_result   = tf.gather_nd(gt_scatter[...,:-1], gather_inds, name  = tensor_name)
    # gt_result   = gt_result[...,:-1]
    # print('    gather_inds shape      : ', gather_inds.get_shape())
    
    # append an index to the end of each row --- commented out 30-04-2018
    # gt_tensor   = tf.concat([gt_tensor, bbox_grid_exp], axis = -1)

    # count based on pred score > 0 (changed from index 0 to -1 on 30-04-2018)   
    # moved outside to call() 12-05-2018
    # gt_cls_cnt  = tf.count_nonzero(gt_result[:,:,:,-1],axis = -1, name = 'gt_cls_count')

    
    print('    ',tensor_name, 'final shape  : ', gt_result.get_shape())

    return  gt_result

              

In [ ]:
##----------------------------------------------------------------------------------------------------------------------          
## 
##----------------------------------------------------------------------------------------------------------------------          
def build_ground_truth_tf(gt_class_ids, gt_bboxes, config):
    # // pass model to TensorBuilder
    batch_size      = config.BATCH_SIZE
    num_classes     = config.NUM_CLASSES
    h, w            = config.IMAGE_SHAPE[:2]
    num_bboxes      = gt_bboxes.shape[1]    # 32 (or 100 -> which we dont use anymore)
    
    # num of bounding boxes is determined by bbox_list.shape[1] instead of config.DETECTION_MAX_INSTANCES
    # use of this routine for both input_gt_boxes, and target_gt_deltas
    if num_bboxes == config.DETECTION_MAX_INSTANCES:
        tensor_name = "gt_tensor_max"
    else:
        tensor_name = "gt_tensor"
        
    print('\n')
    print('  > BUILD_GROUND TRUTH_TF()' )
    print('    gt_class_ids shape : ', gt_class_ids.get_shape())
    print('    gt_bboxes.shape    : ', gt_bboxes.get_shape() )

    #---------------------------------------------------------------------------
    # use the argmaxof each row to determine the dominating (predicted) class
    # mask identifies class_ids > 0 
    #---------------------------------------------------------------------------
    gt_classes_exp = tf.to_float(tf.expand_dims(gt_class_ids ,axis=-1))
    print('    gt_classes_exp shape ', gt_classes_exp.get_shape() )

    ones = tf.ones_like(gt_class_ids)
    zeros= tf.zeros_like(gt_class_ids)
    mask = tf.greater(gt_class_ids , 0)

    gt_scores     = tf.where(mask, ones, zeros)
    # pred_scores      = tf.reduce_max(mrcnn_class ,axis=-1, keep_dims=True)   # (32,)
    gt_scores_exp = tf.to_float(KB.expand_dims(gt_scores, axis=-1))
    print('    gt_scores_exp shape ', gt_scores_exp.get_shape())

    #---------------------------------------------------------------------------
    # create meshgrid to do something 
    #---------------------------------------------------------------------------
    batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32), 
                                         tf.range(num_bboxes, dtype=tf.int32), indexing = 'ij' )
    
    # sequence id is used to preserve the order of rois as passed to this routine
    sequence = gt_scores * (bbox_grid[...,::-1] + 1) 
    sequence = tf.to_float(tf.expand_dims(sequence, axis = -1))   

    # print('    batch_grid shape  ', batch_grid.get_shape())
    # print('    bbox_grid  shape  ', bbox_grid.get_shape())
    # print('    sequence shape    ', sequence.get_shape())

    # obsolete replaced with sequence 12-05-2018    
    # bbox_idx_zeros  = tf.zeros_like(bbox_grid)
    # bbox_idx        = tf.where(mask, bbox_grid , bbox_idx_zeros)
    # bbox_idx        = tf.to_float(tf.expand_dims(bbox_idx, axis = -1))    
    
    # 12-05-2018 (added sequecne)    gt_array  = tf.concat([gt_bboxes, gt_classes_exp, gt_scores_exp], axis=2)
    gt_array        = tf.concat([gt_bboxes, gt_classes_exp, gt_scores_exp, sequence ], axis=2)
    print('    gt_array shape :', gt_array.shape, gt_array.get_shape())

    #------------------------------------------------------------------------------
    # Create indicies to scatter rois out to multi-dim tensor by image id and class
    # resulting tensor is batch size x num_classes x num_bboxes x 7 (num columns)
    #------------------------------------------------------------------------------
    scatter_ind = tf.stack([batch_grid , gt_class_ids, bbox_grid],axis = -1)

    # obsolete 12-05-2018 - replaced num_cols with gt_array.shape[-1]
    # gt_scatter = tf.scatter_nd(scatter_ind, gt_array, [batch_size, num_classes, num_bboxes, num_cols ])
    gt_scatter = tf.scatter_nd(scatter_ind, gt_array, [batch_size, num_classes, num_bboxes, gt_array.shape[-1] ])

    # print('    scatter_ind shape ', scatter_ind.get_shape())
    # print('    gt_scatter shape ', gt_scatter.get_shape())
    
    #-------------------------------------------------------------------------------
    ## sort in each class dimension based on on sequence number (last column)
    # scatter_nd places bboxs in a sparse fashion --- this sort is to place all bboxes
    # at the top of the class bbox array
    #-------------------------------------------------------------------------------

    _ , sort_inds = tf.nn.top_k(tf.abs(gt_scatter[:,:,:,-1]), k=gt_scatter.shape[2])

    # build indexes to gather rows from pred_scatter based on sort order 
    class_grid, batch_grid, bbox_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_bboxes))
    bbox_grid_exp = tf.to_float(tf.expand_dims(bbox_grid, axis = -1))
 
    gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
    gt_result   = tf.gather_nd(gt_scatter[...,:-1], gather_inds, name  = tensor_name)
    # gt_result   = gt_result[...,:-1]
    # print('    gather_inds shape      : ', gather_inds.get_shape())
    
    # append an index to the end of each row --- commented out 30-04-2018
    # gt_tensor   = tf.concat([gt_tensor, bbox_grid_exp], axis = -1)

    # count based on pred score > 0 (changed from index 0 to -1 on 30-04-2018)   
    # moved outside to call() 12-05-2018
    # gt_cls_cnt  = tf.count_nonzero(gt_result[:,:,:,-1],axis = -1, name = 'gt_cls_count')

    
    print('    ',tensor_name, 'final shape  : ', gt_result.get_shape())

    return  gt_result

              

In [ ]:
# with sess.as_default():
    
#     gt_bboxes    = tf.identity(roi_gt_boxes)
#     gt_class_ids = tf.identity(target_class_ids)
#     result = build_ground_truth_tf(gt_class_ids, gt_bboxes, model.config).eval()

In [ ]:
img = 1
print(target_class_ids.shape)
print(target_class_ids[img])
print(result.shape)
print(result[img,:,:15])
print(' roi_gt_boxes')
print(roi_gt_boxes.shape)
print(roi_gt_boxes[img,:15] )

##  `development build_grround_truth_tf ()` 

In [ ]:
sess = KB.get_session()
gt_bboxes    = tf.identity(target_bbox_deltas)
gt_class_ids = tf.identity(target_class_ids)
num_bboxes   = 32

In [ ]:
# // pass model to TensorBuilder
batch_size      = config.BATCH_SIZE
num_classes     = config.NUM_CLASSES
h, w            = config.IMAGE_SHAPE[:2]
num_cols        = 6

# num of bounding boxes is determined by bbox_list.shape[1] instead of config.DETECTION_MAX_INSTANCES
# use of this routine for both input_gt_boxes, and target_gt_deltas
if num_bboxes == config.DETECTION_MAX_INSTANCES:
    tensor_name = "gt_tensor"
else:
    tensor_name = "gt_deltas"

print('\n')
print('  > BUILD_GROUND TRUTH_TF()' )
print('    gt_class_ids shape : ', KB.int_shape(gt_class_ids), '    gt_bboxes.shape  : ', gt_bboxes.get_shape() )

In [ ]:
#---------------------------------------------------------------------------
# use the argmaxof each row to determine the dominating (predicted) class
#---------------------------------------------------------------------------
# gt_classes     = gt_class_ids    # batch_size x max gt detections
gt_classes_exp = tf.to_float(tf.expand_dims(gt_class_ids ,axis=-1))
print('    gt_classes_exp shape ', gt_classes_exp.get_shape() )

ones = tf.ones_like(gt_class_ids)
zeros= tf.zeros_like(gt_class_ids)
mask = tf.greater(gt_class_ids , 0)
print(mask.shape)
print(mask.eval(session=sess))
gt_scores  =  tf.where(mask, ones, zeros)
# pred_scores      = tf.reduce_max(mrcnn_class ,axis=-1, keep_dims=True)   # (32,)
gt_scores_exp = tf.to_float(tf.expand_dims(gt_scores, axis=-1))
print('    gt_scores shape ', gt_scores.get_shape())
print(gt_scores.eval(session=sess))    


In [ ]:
#---------------------------------------------------------------------------
# create meshgrid to do something 
#---------------------------------------------------------------------------
batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size    , dtype=tf.int32), 
                                    tf.range(num_bboxes, dtype=tf.int32), indexing = 'ij' )

# sequence id is used to preserve the order of rois as passed to this routine
sequence = gt_scores * (bbox_grid[...,::-1] + 1) 
sequence = tf.to_float(tf.expand_dims(sequence, axis = -1))   

print('    batch_grid shape  ', batch_grid.get_shape())
print(batch_grid.eval(session=sess))
print('    bbox_grid  shape  ', bbox_grid.get_shape())
print(bbox_grid.eval(session=sess))
print('    sequence shape    ', sequence.get_shape())
print(sequence.eval(session=sess)) 

gt_array        = tf.concat([gt_bboxes, gt_classes_exp, gt_scores_exp, sequence ], axis=2)

print('    gt_array shape    ', gt_array.get_shape())
print(gt_array.eval(session=sess)) 
print(gt_array.shape[-1])    
# print(gt_array[:,:10].eval(session=sess))


In [ ]:
#------------------------------------------------------------------------------
# Create indicies to scatter rois out to multi-dim tensor by image id and class
# resulting tensor is batch size x num_classes x num_detections x 7 (num columns)
#------------------------------------------------------------------------------
scatter_ind = tf.stack([batch_grid , gt_class_ids, bbox_grid],axis = -1)
print('    scatter_ind shape ', scatter_ind.get_shape())
print(scatter_ind.eval(session=sess))


gt_scatter = tf.scatter_nd(scatter_ind, gt_array, [batch_size, num_classes, num_bboxes, gt_array.shape[-1] ])


# print('-- stack results ----')
print('    scatter_ind shape ', scatter_ind.get_shape())
# print(scatter_ind.eval())
print('    gt_scatter shape ', gt_scatter.get_shape())

print(gt_scatter[1,:,:,:].eval(session=sess))

In [ ]:
#-------------------------------------------------------------------------------
## sort in each class dimension based on y2 (column 1)
# scatter_nd places bboxs in a sparse fashion --- this sort is to place all bboxes
# at the top of the class bbox array
#-------------------------------------------------------------------------------
_ , sort_inds = tf.nn.top_k(gt_scatter[:,:,:,-1] , k=gt_scatter.shape[2])

# build indexes to gather rows from pred_scatter based on sort order 
class_grid, batch_grid, bbox_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_bboxes))
bbox_grid_exp = tf.to_float(tf.expand_dims(bbox_grid, axis = -1))

print('    build gathering indexes to use in sorting -------')    
print('    sort inds shape : ', sort_inds.get_shape())
print('    class_grid  shape ', class_grid.get_shape())
# print(class_grid.eval())
print('    batch_grid  shape ', batch_grid.get_shape())
# print(batch_grid.eval())
print('    bbox_grid   shape ', bbox_grid.get_shape() , ' bbox_grid_exp shape ', bbox_grid_exp.get_shape())
# print(bbox_grid.eval())

gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
gt_tensor   = tf.gather_nd(gt_scatter, gather_inds, name  = tensor_name)[...,:-1]
print('    gather_inds shape      : ', gather_inds.get_shape())
print('    gt_tensor (gathered)   : ', gt_tensor.get_shape())

In [ ]:
print(gt_tensor[2].eval(session=sess))
print(target_bbox_deltas[2])

In [ ]:
# with sess.as_default():
#     np_gt_array = gt_array.eval()
#     np_gt_result = gt_result.eval()
# np.set_printoptions(precision=4, threshold=10000, linewidth=120, suppress=True)
# print(np_gt_array.shape, np_gt_result.shape)
# print(np_gt_result)

In [ ]:
# append an index to the end of each row --- commented out 30-04-2018
# gt_tensor   = tf.concat([gt_tensor, bbox_grid_exp], axis = -1)

# count based on pred score > 0 (changed from index 0 to -1 on 30-04-2018)    
gt_cls_cnt  = tf.count_nonzero(gt_tensor[:,:,:,-1],axis = -1, name = 'gt_cls_count')

print('    final gt_tensor shape  : ', gt_tensor.get_shape())
print('    final gt_cls_cnt shape : ', gt_cls_cnt.get_shape())
print('    ', gt_cls_cnt.eval(session=sess))
print('    complete')

# return  [gt_tensor, gt_cls_cnt] 

In [ ]:
print(input_rpn_bbox.shape)
print(target_bbox_deltas[0])
print(gt_bboxes[0])
print(gt_class_ids[0])
print(gt_tensor[0,2].eval(session=sess))


In [ ]:
print(target_class_ids.shape)
sess = KB.get_session()
with sess.as_default():
    print(tf.where(target_class_ids > 0).eval())
# target_class_ids[0]
# target_bbox_deltas[0]

##  `old build_ground_truth_tf ()  (as of 18-05-2018)` 

In [ ]:
##----------------------------------------------------------------------------------------------------------------------          
##  build°groundtruth°_tf as of 18-05-2018
##----------------------------------------------------------------------------------------------------------------------          
def build_ground_truth_tf(gt_class_ids, gt_bboxes, num_bboxes, config):
    # // pass model to TensorBuilder
    batch_size      = config.BATCH_SIZE
    num_classes     = config.NUM_CLASSES
    h, w            = config.IMAGE_SHAPE[:2]
    num_cols        = 7
    
    # num of bounding boxes is determined by bbox_list.shape[1] instead of config.DETECTION_MAX_INSTANCES
    # use of this routine for both input_gt_boxes, and target_gt_deltas
    if num_bboxes == config.DETECTION_MAX_INSTANCES:
        tensor_name = "gt_tensor"
    else:
        tensor_name = "gt_deltas"
        
    print('\n')
    print('  > BUILD_GROUND TRUTH_TF()' )
    print('    gt_class_ids shape : ', gt_class_ids.get_shape())
    print('    gt_bboxes.shape    : ', gt_bboxes.get_shape() )


    #---------------------------------------------------------------------------
    # use the argmaxof each row to determine the dominating (predicted) class
    # mask identifies class_ids > 0 
    #---------------------------------------------------------------------------
    gt_classes_exp = tf.to_float(tf.expand_dims(gt_class_ids ,axis=-1))
    print('    gt_classes_exp shape ', gt_classes_exp.get_shape() )

    ones = tf.ones_like(gt_class_ids)
    zeros= tf.zeros_like(gt_class_ids)
    mask = tf.greater(gt_class_ids , 0)

    gt_scores     = tf.where(mask, ones, zeros)
    # pred_scores      = tf.reduce_max(mrcnn_class ,axis=-1, keep_dims=True)   # (32,)
    gt_scores_exp = tf.to_float(KB.expand_dims(gt_scores, axis=-1))
    print('    gt_scores_exp shape ', gt_scores_exp.get_shape())

    #---------------------------------------------------------------------------
    # create meshgrid to do something 
    #---------------------------------------------------------------------------
    batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32), 
                                         tf.range(num_bboxes, dtype=tf.int32), indexing = 'ij' )
    
    # sequence id is used to preserve the order of rois as passed to this routine
    sequence = gt_scores * (bbox_grid[...,::-1] + 1) 
    sequence = tf.to_float(tf.expand_dims(sequence, axis = -1))   

    # print('    batch_grid shape  ', batch_grid.get_shape())
    # print('    bbox_grid  shape  ', bbox_grid.get_shape())
    # print('    sequence shape    ', sequence.get_shape())

    # obsolete replaced with sequence 12-05-2018    
    # bbox_idx_zeros  = tf.zeros_like(bbox_grid)
    # bbox_idx        = tf.where(mask, bbox_grid , bbox_idx_zeros)
    # bbox_idx        = tf.to_float(tf.expand_dims(bbox_idx, axis = -1))    
    
    # 12-05-2018 (added sequecne)    gt_array        = tf.concat([gt_bboxes, gt_classes_exp, gt_scores_exp], axis=2)
    gt_array        = tf.concat([gt_bboxes, gt_classes_exp, gt_scores_exp, sequence ], axis=2)
    print('    gt_array shape :', gt_array.shape, gt_array.get_shape())    
    
    #------------------------------------------------------------------------------
    # Create indicies to scatter rois out to multi-dim tensor by image id and class
    # resulting tensor is batch size x num_classes x num_bboxes x 7 (num columns)
    #------------------------------------------------------------------------------
    scatter_ind = tf.stack([batch_grid , gt_class_ids, bbox_grid],axis = -1)
#     gt_scatter = tf.scatter_nd(scatter_ind, gt_array, [batch_size, num_classes, num_bboxes, num_cols ])
    gt_scatter = tf.scatter_nd(scatter_ind, gt_array, [batch_size, num_classes, num_bboxes, gt_array.shape[-1] ])
    # print('    scatter_ind shape ', scatter_ind.get_shape())
    # print('    gt_scatter shape ', gt_scatter.get_shape())
    
    #-------------------------------------------------------------------------------
    ## sort in each class dimension based on on sequence number (last column)
    # scatter_nd places bboxs in a sparse fashion --- this sort is to place all bboxes
    # at the top of the class bbox array
    #-------------------------------------------------------------------------------

    _ , sort_inds = tf.nn.top_k(tf.abs(gt_scatter[:,:,:,-1]), k=gt_scatter.shape[2])

    # build indexes to gather rows from pred_scatter based on sort order 
    class_grid, batch_grid, bbox_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_bboxes))
    bbox_grid_exp = tf.to_float(tf.expand_dims(bbox_grid, axis = -1))
 
    gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
    gt_result   = tf.gather_nd(gt_scatter[...,:-1], gather_inds, name  = tensor_name)
    # gt_result   = gt_result[...,:-1]
    # print('    gather_inds shape      : ', gather_inds.get_shape())
    
    # append an index to the end of each row --- commented out 30-04-2018
    # gt_tensor   = tf.concat([gt_tensor, bbox_grid_exp], axis = -1)

    # count based on pred score > 0 (changed from index 0 to -1 on 30-04-2018)   
    # moved outside to call() 12-05-2018
    # gt_cls_cnt  = tf.count_nonzero(gt_result[:,:,:,-1],axis = -1, name = 'gt_cls_count')
    print('    ',tensor_name, 'final shape  : ', gt_result.get_shape())

    return  gt_result
    #-------------------------------------------------------------------------------